In [372]:
# pip install dash-bootstrap-components

In [373]:
import pandas as pd, plotly.express as px
import dash_bootstrap_components as dbc
from dash import dcc, html, Dash, Input, Output

In [374]:

df = pd.read_csv('consdata.csv')  #Reading dataset
df['Date Time Served'] = pd.to_datetime(df['Date Time Served'])
df['Opening Balance (ml)'] = pd.to_numeric(df['Opening Balance (ml)'])
df['Purchase (ml)'] = pd.to_numeric(df['Purchase (ml)'])
df['Consumed (ml)'] = pd.to_numeric(df['Consumed (ml)'])
df['Closing Balance (ml)'] = pd.to_numeric(df['Closing Balance (ml)'])
df['Day'] = df['Date Time Served'].dt.day
df['Month'] = df['Date Time Served'].dt.month
df['Year'] = df['Date Time Served'].dt.year
df['Week'] = df['Date Time Served'].dt.dayofweek
df['Time (Hour)'] = df['Date Time Served'].dt.hour
df['Time (Mins)'] = df['Date Time Served'].dt.minute
dfCopy = df.sort_values(['Year', 'Month', 'Day', 'Time (Hour)', 'Time (Mins)'], ascending=True).copy()
# dfCopy

df2023 = dfCopy[dfCopy['Year'].astype('str').str.contains('2023')].copy()
# df2023

dfNone = dfCopy[dfCopy['Bar Name'].str.contains('None')].copy()
df2023


,Date Time Served,Bar Name,Alcohol Type,Brand Name,Opening Balance (ml),Purchase (ml),Consumed (ml),Closing Balance (ml),Day,Month,Year,Week,Time (Hour),Time (Mins)
1,2023-01-01 10:07:00,Smith's Bar,Wine,Yellow Tail,1344.37,0.0,0.00,1344.37,1,1,2023,6,10,7
5,2023-01-01 10:28:00,Brown's Bar,Vodka,Grey Goose,2560.44,0.0,0.00,2560.44,1,1,2023,6,10,28
2,2023-01-01 11:26:00,Johnson's Bar,Vodka,Grey Goose,1034.28,0.0,0.00,1034.28,1,1,2023,6,11,26
13,2023-01-01 11:32:00,Anderson's Bar,Whiskey,Jim Beam,872.08,0.0,0.00,872.08,1,1,2023,6,11,32
3,2023-01-01 13:53:00,Johnson's Bar,Beer,Coors,2194.53,0.0,0.00,2194.53,1,1,2023,6,13,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6551,2023-12-31 19:29:00,Taylor's Bar,Whiskey,Jim Beam,859.78,0.0,349.75,510.03,31,12,2023,6,19,29
6550,2023-12-31 19:58:00,Taylor's Bar,Rum,Malibu,2026.72,0.0,220.20,1806.52,31,12,2023,6,19,58
6553,2023-12-31 20:19:00,Anderson's Bar,Wine,Yellow Tail,758.06,0.0,129.41,628.65,31,12,2023,6,20,19
6548,2023-12-31 20:47:00,Taylor's Bar,Wine,Sutter Home,0.00,0.0,0.00,0.00,31,12,2023,6,20,47


In [375]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc.icons.FONT_AWESOME] )

In [376]:

app.layout = ([

    dbc.Row([
        dbc.CardBody([
            dbc.Card([
                dbc.Col([
                    html.H1('Figures for the following dataset!',className='text-center my-5'),
                ])
            ])
        ])
    ]),
#Graph of Time(as a changable) to Purhcase(ml) for all the bars
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H3('This is a graph of how the bars are related to the month of that year', className='card-title'),
                    dcc.Dropdown(id = 'monthSelected',
                                 options=[{'label':month, 'value':month} for month in df2023['Month'].unique()],
                                 value=None,
                                 style={'color': 'Black', 'font-size': 15},
                                 placeholder='Select a month'),
                    dcc.Dropdown(id = 'barSelected',
                                 options=[{'label':bar, 'value':bar} for bar in df2023['Bar Name'].unique()],
                                 value=None,
                                 style={'color': 'Black', 'font-size': 15},
                                 placeholder='Select a Bar'),
                    dcc.Graph(id = 'barToMonth')

                ])
            ])
        ], width='auto')
    ], justify="center"),

#The following will contain graphs with week of the day as a list and the
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H3('This is a graph of how the bars are related to the day to the week ', className='card-title'),
                    dcc.Dropdown(id = 'weekSelected',
                                 options=[{'label':dayOfWeek, 'value':dayOfWeek} for dayOfWeek in df2023['Week'].unique()],
                                 value=None,
                                 style={'color': 'Black', 'font-size': 15},
                                 placeholder='Select a day of the week, 0 - Monday, till 6 - Sunday'),
                    dcc.Graph(id = 'barToDayoftheWeek')

                ])
            ])
        ], width='auto')
    ], justify="center"),

    # dbc.Row([
    #     dbc.Col([
    #         dbc.Card([
    #             dbc.CardBody([
    #                 html.H3('This is a graph of how the bars are related to the month of that year', className='card-title'),
    #                 dcc.Dropdown(id = 'monthSelected',
    #                              options=[{'label':month, 'value':month} for month in df2023['Month'].unique()],
    #                              value=None,
    #                              placeholder='Select a month'),
    #                 dcc.Graph(id = 'barToMonth')
    #
    #             ])
    #         ])
    #     ], width='auto')
    # ], justify="center"),

])

#For Month
@app.callback(
    Output('barToMonth', 'figure'),
    [Input('monthSelected', 'value'),
     Input('barSelected', 'value')]
)
def update_month_or_bar(monthOfTheYear, barSelected):
    f1df2023 = df2023[df2023['Purchase (ml)'] != 0].copy()
    if monthOfTheYear:
        f1df2023 = f1df2023[f1df2023['Month'] == monthOfTheYear]
        if barSelected:
            f1df2023 = f1df2023[f1df2023['Bar Name'] == barSelected]
    elif barSelected:
        f1df2023 = f1df2023[f1df2023['Bar Name'] == barSelected]
        if monthOfTheYear:
            f1df2023 = f1df2023[f1df2023['Month'] == monthOfTheYear]
    else:
        f1df2023 = f1df2023

    figBarYearToMonth = px.histogram(f1df2023,x='Bar Name', y='Purchase (ml)', color='Alcohol Type', barmode='group', color_discrete_sequence=px.colors.qualitative.G10, template='plotly_dark', title='Each month to all bars to Purchase(ml)')

    return figBarYearToMonth

#For Day of the Week
@app.callback(
    Output('barToDayoftheWeek', 'figure'),
    Input('weekSelected', 'value')
)
def update_week(weekDaySelected):
    f2df2023 = df2023[df2023['Purchase (ml)'] != 0].copy()
    if weekDaySelected:
        f2df2023 = f2df2023[f2df2023['Week'] == weekDaySelected]
    else:
        f2df2023 = f2df2023

    figWeekofDay = px.histogram(f2df2023,x='Bar Name', y='Purchase (ml)', color='Bar Name', barmode='group', color_discrete_sequence=px.colors.qualitative.G10, template='plotly_dark', title='Each day of the week to all bars to Purchase(ml)')

    return figWeekofDay


In [377]:
if __name__ == '__main__':
    app.run(port=4050)